In [1]:
import sys
import os
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")

atlas_path = "C:/Users/natha/OneDrive/Desktop/C++/Atlas/x64/Debug"
sys.path.append(atlas_path)


from AtlasPy.core import Hydra, Portfolio, Strategy
from AtlasPy.ast import *

In [2]:
exchange_path = "C:/Users/natha/OneDrive/Desktop/C++/Atlas/AtlasPy/src/exchangeVBT"
strategy_id = "test_strategy"
exchange_id = "test_exchange"
portfolio_id = "test_portfolio"

hydra = Hydra()
intial_cash = 100.0
exchange = hydra.addExchange(exchange_id, exchange_path, "%Y-%m-%d %H:%M:%S")
portfolio = hydra.addPortfolio(portfolio_id, exchange, intial_cash)
hydra.build()

In [10]:
st = time.time()
fast_n = 6
slow_n = 12
windows = np.arange(5, 11)

close = AssetReadNode.make("Close", 0, exchange)
fast_ma = MeanObserverNode(
    close,
    fast_n
)
slow_ma = MeanObserverNode(
    close,
    slow_n
)

spread = AssetOpNode.make(fast_ma, slow_ma, AssetOpType.SUBTRACT)
spread_filter = ExchangeViewFilter(ExchangeViewFilterType.GREATER_THAN, 0.0, None)
exchange_view = ExchangeViewNode.make(exchange, spread, spread_filter)

fast_ma_dim = GridDimensionObserver.make(
    name="Fast MA",
    dimension_values=windows,
    observer_base=fast_ma,
    observer_child=spread,
    swap_addr=spread.getSwapLeft()
)
slow_ma_dim = GridDimensionObserver.make(
    name="Slow MA",
    dimension_values=windows,
    observer_base=slow_ma,
    observer_child=spread,
    swap_addr=spread.getSwapRight()
)

allocation = AllocationNode.make(exchange_view)
strategy_node = StrategyNode.make(allocation, portfolio)
strategy = hydra.addStrategy(Strategy(strategy_id, strategy_node, 1.0), True)
grid = strategy.setGridDimmensions((fast_ma_dim, slow_ma_dim))
grid.enableTracerHistory(TracerType.NLV)
strategy.enableTracerHistory(TracerType.NLV)
hydra.run()

nlv = strategy.getHistory(TracerType.NLV)
returns = nlv[-1] / nlv[0] - 1

et = time.time()
print(f'Elapsed time: {1000*(et-st):.2f} ms')
print(f'Total return: {returns}')

Elapsed time: 713.77 ms
Total return: 1.2212700459014085


In [11]:
nlv = strategy.getHistory(TracerType.NLV)
timestamps = pd.to_datetime(exchange.getTimestamps())
returns = nlv[-1] / nlv[0] - 1
print("Returns: ", returns)

Returns:  1.2212700459014085


In [12]:
rows = grid.rows()
cols = grid.cols()

pairs = []
nlvs = []
returns_dict = {}
for i in range(rows):
    for j in range(cols):
        if i >= j:
            continue

        pairs.append((windows[i], windows[j]))
        tracer = grid.getTracer(i, j)
        nlv = tracer.getHistory(TracerType.NLV)
        returns = nlv[-1] / nlv[0] - 1
        returns_dict[pairs[-1]] = returns
        nlvs.append(nlv)

returns_dict

{(5, 6): 2.0959537612945423,
 (5, 7): 1.647424549756637,
 (5, 8): 4.704818981171171,
 (5, 9): 2.629302608885226,
 (5, 10): 2.098764661230769,
 (6, 7): 4.468969951287569,
 (6, 8): 5.106150380943816,
 (6, 9): 3.5521623341988864,
 (6, 10): 2.211440777292788,
 (7, 8): 7.70781073250849,
 (7, 9): 1.3559104370733963,
 (7, 10): 2.2361199974943746,
 (8, 9): 0.23387927412516785,
 (8, 10): 1.8206289085586387,
 (9, 10): 3.5219102505975597}

In [6]:
avg = np.mean(list(returns_dict.values()))
print(f'Average return: {avg}')

Average return: 3.8471186759284643
